# Проект: Интерактивная текстовая игра — *«Трактир \u00abЛогос\u00bb»*

Переработанная версия вашего сюжета, оформленная по критериям: функции, циклы (`while`/`for`), списки/множества/словари, запись в файл `results.txt`.

**Как запустить:** выполните ячейку с кодом ниже, затем в новой ячейке вызовите `play()`.


In [ ]:
# Интерактивная текстовая игра: "Трактир «Логос»"
# ------------------------------------------------
# Что проверяется этим проектом согласно критериям:
# - Условные конструкции (if/elif/else) — переходы между сценами и концовками.
# - Циклы: while — основной игровой цикл и проверки ввода; for — вывод вариантов/номеров.
# - Списки/множества/словари: 
#     * список `path_log` (методы .append(), .extend());
#     * множества `inventory`, `visited` (методы .add(), .update(), .discard());
#     * готовые словари `SCENES`, `ENDINGS`;
#     * создаваемый во время игры словарь `visit_count` (учёт посещений).
# - Функции: `show_scene`, `save_results`, `normalize`.
# - Запись в файл: `results.txt` с ходом игры и концовкой.
# - Без импортов (0 штрафа за библиотеки).
#
# Источник: переработанная и структурированная версия вашего сюжета про трактир «Логос».

# --------------------------- ДАННЫЕ И СЦЕНЫ ---------------------------

SCENES = {
    'start': {
        'title': 'Вьюга и двери «Логоса»',
        'desc': [
            'Вы заблудились в лесу. Бушует метель.',
            'Наконец добираетесь до трактира «Логос». Дверь тяжёлая, тёплый свет манит внутрь.',
            'Голос бармена: «Добро пожаловать! Я — Айзек. Чай? Ром? Или вас привела другая причина?»'
        ],
        'options': [
            ('Выпить чаю', 'tea!'),
            ('Выпить рома', 'rum!'),
            ('Просто отдохнуть у камина', 'rest!'),
            ('Другая причина', 'other_reason')
        ]
    },
    'other_reason': {
        'title': 'Барная стойка',
        'desc': [
            'Айзек: «Вы случайно не для заселения пришли?»'
        ],
        'options': [
            ('Да, хочу номер', 'checkin_yes'),
            ('Нет, нужна помощь', 'checkin_no')
        ]
    },
    'checkin_yes': {
        'title': 'Выбор комнаты',
        'desc': [
            'Айзек: «Свободны четыре номера. Учтите: нечётные — посредственные, чётные — полны сюрпризов.»'
        ],
        # Переход на служебную сцену, где будет for по списку номеров
        'options': [
            ('Посмотреть номера', 'rooms_list')
        ]
    },
    'rooms_list': {
        'title': 'Номера',
        'desc': [
            'На табло мерцают номера: 0, 1, 2, 3. Выберите номер.'
        ],
        # тут ввод вручную числом; варианты не выводим, а печатаем список через for
        'options': []
    },
    'even_room_pull': {
        'title': 'Белая комната',
        'desc': [
            'Комната залита белым светом. Вас тянет внутрь, дверь уходит из рук...'
        ],
        'options': [
            ('Далее', 'stranger_q_faith')
        ]
    },
    'stranger_q_faith': {
        'title': 'Незнакомец',
        'desc': [
            'В четырех стенах вас встречает незнакомец.',
            'Незнакомец: «Два вопроса. В Бога веришь?»'
        ],
        'options': [
            ('Да', 'faith_yes'),
            ('Нет', 'strangled1!')
        ]
    },
    'faith_yes': {
        'title': 'Незнакомец (ч.2)',
        'desc': [
            'Незнакомец: «Молилась ли ты на ночь, Дездемона?»'
        ],
        'options': [
            ('Да', 'strangled2!'),
            ('Нет', 'poe_raven')
        ]
    },
    'poe_raven': {
        'title': 'Красная мантия',
        'desc': [
            'Существо в красной мантии: «Что говорил ворон Эдгара По, сидя на бюсте Паллады?»'
        ],
        'options': [
            ('«никогда»', 'dust1!'),
            ('«никогда больше»', 'red_death_identity')
        ]
    },
    'red_death_identity': {
        'title': 'Лик под маской',
        'desc': [
            'Существо: «Кто я?»'
        ],
        'options': [
            ('Смерть', 'dust2!'),
            ('Маска Красной Смерти', 'dante_quote')
        ]
    },
    'dante_quote': {
        'title': 'Чёрный плащ',
        'desc': [
            'Человек в чёрном: «Оставь надежду, всяк сюда входящий...» Откуда эта цитата?'
        ],
        'options': [
            ('«Фауст»', 'noose!'),
            ('«Божественная комедия»', 'phantom_boxes')
        ]
    },
    'phantom_boxes': {
        'title': 'Две шкатулки',
        'desc': [
            'На одной — золотой скорпион, на другой — серебряный кузнечик. «Выбирай.»'
        ],
        'options': [
            ('Кузнечик', 'sky_box!'),
            ('Скорпион', 'scorpion_box_success!')
        ]
    },
    'checkin_no': {
        'title': 'Помощь и дорога',
        'desc': [
            'Вы признаётесь: заблудились в метели. Айзек: «Оставайтесь.',
            'Только помогите на кухне — посуды гора, работники не справляются.»'
        ],
        'options': [
            ('Помочь на кухне', 'help_return!')
        ]
    }
}

# Кодовые названия концовок и тексты
ENDINGS = {
    'tea': 'Вы согрелись чаем и наблюдали за вьюгой за окном. Ночь встретила вас мягко. (Спокойная концовка)',
    'rum': 'Вы переборщили с ромом и рухнули на пол. Утро будет немилосердным. (Плохая концовка)',
    'rest': 'Вы уснули у камина. Сны унесли тревоги. (Нейтральная концовка)',
    'modest_room': 'Вы выбрали скромный номер и уснули без приключений. (Нейтральная концовка)',
    'strangled1': '«Тогда второго вопроса не будет», — шепчет незнакомец. Тьма сжимает горло. (Плохая концовка)',
    'strangled2': '«Тогда по сюжету», — усмешка. Руки сомкнулись на шее. (Плохая концовка)',
    'dust1': 'Ледяные ладони — и вы рассыпались в пыль. (Плохая концовка)',
    'dust2': 'Ошибка стоит дорого: прикосновение — и вы исчезаете пылью. (Плохая концовка)',
    'noose': 'Пенджабская удавка срывает дыхание. Смех звенит в ушах. (Плохая концовка)',
    'sky_box': 'Комната взлетает к небесам и взрывается. Ничто не остаётся. (Плохая концовка)',
    'scorpion_box_success': 'Маска падает, удавка срывается, и вы уходите к спасительной двери. Дом встречает через мгновение. (Хорошая концовка)',
    'help_return': 'Вы отмыли «весь Эверест» тарелок, уснули — и очнулись дома, далеко от «Логоса». (Тёплая концовка)'
}

# ------------------------------- СОСТОЯНИЕ -------------------------------

path_log = []            # список действий игрока
inventory = set()        # демонстрационно — сюда кладём условные «ключи»
visited = set()          # посещённые сцены
visit_count = {}         # будет заполняться во время игры (созданный словарь)

# ------------------------------- ВСПОМОГАТЕЛЬНОЕ -------------------------------

def normalize(s):
    """Приводим пользовательский ввод к нижнему регистру и обрезаем пробелы."""
    return (s or '').strip().lower()

def save_results(ending_code):
    """Запись результатов в results.txt: концовка, инвентарь, посещения, путь."""
    try:
        with open('results.txt', 'a', encoding='utf-8') as f:
            f.write('=== Игровая сессия: «Логос» ===\n')
            f.write('ИТОГ: ' + ENDINGS.get(ending_code, 'Неизвестная концовка') + '\n')
            inv = sorted(list(inventory))
            vis = sorted(list(visited))
            f.write('Инвентарь: ' + (', '.join(inv) if inv else 'пусто') + '\n')
            f.write('Посещённые сцены: ' + (', '.join(vis) if vis else '—') + '\n')
            for i, step in enumerate(path_log, 1):
                f.write(f'{i:02d}. {step}\n')
            f.write('\n')
    except Exception as e:
        print('Ошибка записи:', e)

# ------------------------------- ЛОГИКА СЦЕН -------------------------------

def show_scene(scene_key):
    """Показывает сцену, печатает варианты и возвращает следующий scene_key или 'ending!'"""
    visit_count[scene_key] = visit_count.get(scene_key, 0) + 1  # создаём/увеличиваем запись в словаре
    visited.add(scene_key)                                      # метод множества .add()

    scene = SCENES[scene_key]

    print('\n' + '='*60)
    print(scene['title'])
    print('-'*60)
    # Вывод описания (список строк)
    for line in scene['desc']:
        print(line)
    print('-'*60)

    # Специальная отрисовка номеров комнат со списком и циклом for
    if scene_key == 'rooms_list':
        rooms = [0, 1, 2, 3]
        print('Доступные номера:')
        for r in rooms:
            print('  -', r)
        choice = input('Введите номер (0/1/2/3): ')
        path_log.append(f'[rooms_list] ввод: {choice}')
        # Проверяем выбор и направляем
        if choice in ('1', '3'):
            return 'modest_room!'
        elif choice in ('0', '2'):
            inventory.update({'ключ-комнаты'})   # демонстрация .update()
            return 'even_room_pull'
        else:
            print('Айзек улыбается: «Похоже, пальцы ещё мёрзнут. Попробуем снова.»')
            return 'rooms_list'

    # Обычные варианты
    options = list(scene['options'])  # копия

    # При повторном визите добавим «подсказку» — демонстрация .extend()
    if visit_count[scene_key] > 1:
        path_log.extend([f'Повторный визит: {scene_key}', 'Замечены новые детали.'])

    # Печать вариантов выбора:
    if options:
        print('Ваши варианты:')
        for i, (label, _) in enumerate(options, 1):
            print(f'  {i}. {label}')
        # Чтение корректного номера — цикл while
        idx = None
        while True:
            raw = input('Введите номер варианта: ').strip()
            if raw.isdigit():
                n = int(raw)
                if 1 <= n <= len(options):
                    idx = n - 1
                    break
            print('Некорректный ввод. Попробуйте снова.')
        label, target = options[idx]
        path_log.append(f'[{scene_key}] -> {label}')

        # Маркер концовки
        if target.endswith('!'):
            return target
        return target

    # Если вариантов нет — безопасный возврат
    return 'start'

# ------------------------------- ИГРОВОЙ ЦИКЛ -------------------------------

def play():
    print('Трактир «Логос» ждёт. Снимите с пальцев лёд и сделайте выбор.')
    current = 'start'
    while True:
        nxt = show_scene(current)

        if isinstance(nxt, str) and nxt.endswith('!'):
            ending = nxt[:-1]
            path_log.append('ИТОГ: ' + ENDINGS.get(ending, ending))
            print('\n' + '='*60)
            print('КОНЕЦ ИСТОРИИ')
            print(ENDINGS.get(ending, 'Неизвестная концовка'))
            print('='*60)
            save_results(ending)
            break
        else:
            current = nxt

# Подсказка: запустите игру командой ниже в отдельной ячейке:
# play()

## Быстрый старт
```python
play()
```
Файл `results.txt` появится/дополнится в той же папке.
